<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qworld.net" target="_blank"><img src="../images/qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by <a href="https://iitis.pl/pl/person/aglos" target="_blank"  >Adam Glos</a> and Özlem Salehi
        </td>        
</tr></table>

<table width="100%"><tr><td style="color:#bbbbbb;background-color:#ffffff;font-size:11px;font-style:italic;text-align:right;">This cell contains some macros. If there is a problem with displaying mathematical formulas, please run this cell to load these macros. </td></tr></table>

$ \renewcommand{\bra}[1]{\langle #1|} $
$ \renewcommand{\ket}[1]{|#1\rangle} $
$ \renewcommand{\braket}[2]{\langle #1|#2\rangle} $
$ \renewcommand{\dot}[2]{ #1 \cdot #2} $
$ \renewcommand{\biginner}[2]{\left\langle #1,#2\right\rangle} $
$ \renewcommand{\mymatrix}[2]{\left( \begin{array}{#1} #2\end{array} \right)} $
$ \renewcommand{\myvector}[1]{\mymatrix{c}{#1}} $
$ \renewcommand{\myrvector}[1]{\mymatrix{r}{#1}} $
$ \renewcommand{\mypar}[1]{\left( #1 \right)} $
$ \renewcommand{\mybigpar}[1]{ \Big( #1 \Big)} $
$ \renewcommand{\sqrttwo}{\frac{1}{\sqrt{2}}} $
$ \renewcommand{\dsqrttwo}{\dfrac{1}{\sqrt{2}}} $
$ \renewcommand{\onehalf}{\frac{1}{2}} $
$ \renewcommand{\donehalf}{\dfrac{1}{2}} $
$ \renewcommand{\hadamard}{ \mymatrix{rr}{ \sqrttwo & \sqrttwo \\ \sqrttwo & -\sqrttwo }} $
$ \renewcommand{\vzero}{\myvector{1\\0}} $
$ \renewcommand{\vone}{\myvector{0\\1}} $
$ \renewcommand{\vhadamardzero}{\myvector{ \sqrttwo \\  \sqrttwo } } $
$ \renewcommand{\vhadamardone}{ \myrvector{ \sqrttwo \\ -\sqrttwo } } $
$ \renewcommand{\myarray}[2]{ \begin{array}{#1}#2\end{array}} $
$ \renewcommand{\X}{ \mymatrix{cc}{0 & 1 \\ 1 & 0}  } $
$ \renewcommand{\Z}{ \mymatrix{rr}{1 & 0 \\ 0 & -1}  } $
$ \renewcommand{\Htwo}{ \mymatrix{rrrr}{ \frac{1}{2} & \frac{1}{2} & \frac{1}{2} & \frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & \frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} \\ \frac{1}{2} & -\frac{1}{2} & -\frac{1}{2} & \frac{1}{2} } } $
$ \renewcommand{\CNOT}{ \mymatrix{cccc}{1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0} } $
$ \renewcommand{\norm}[1]{ \left\lVert #1 \right\rVert } $
$ \renewcommand{\pstate}[1]{ \lceil  #1   \rfloor } $

#  Grover Algorithm for Max-Cut Problem

Finally, we are ready to implement the Grover algorithm for the Max-Cut problem. 

For the initialization, we will apply Hadamard gate to the qubits representing the color of each vertex. We know how to implement the Grover diffusion operator. Now we need to implement an oracle which will "mark" the colorings in which there are $k$ edges whose endpoints are colored using different colors.

For the oracle, the procedure goes as follows:

* Implement edge checking for each edge (check whether an edge has endpoints with different colors).
* Sum the outputs of edge checking step.
* Check whether the sum is equal to (or greater) $k$ and save the flag on the auxillary qubit.
* Apply $Z$ gate on the auxillary qubit.

The oracle works for the decision version of the problem which checks whether there exists a coloring such that there are at least $k$ edges whose endpoints are colored using different colors. How to find the maximum size for the cut?

Suppose that $k$ is a three-digit number $b_2b_1b_0$. First, we should check whether there is a coloring such that $b_2$ is equal to 1. If it is, then we check if there is a coloring such that $b_2=1$ and $b_1=1$. Otherwise, we should check for the coloring such that $b_2=0$ and $b_1$=1. This way, by doing the binary search over binary representation, we can finally find the cut of maximal size.

Let us consider the following graph.
<img src="../images/path3.png" width="25%" align="center">

We will check whether there is a coloring such that two edges connect vertices with a different color. To check that we will verify, whether for number $b_1b_0$, $b_1$ is set to 1. We will use:
* Three qubits (0-2) for vertices.
* Two qubits (3-4) for edges.
* Two qubits (5-6) for storing the sum.
* Single qubit (7) for the auxillary qubit.

Since qubit 7 stores the flag, that is whether $b_1$ is set to 1 or not, we will apply the $Z$ gate on that qubit.

Below we present the final Grover algorithm. Note that there are two such solutions, so we apply the oracle only once.

In [2]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

def oracle_computation():
    #edge checking
    yield CX(qq[0], qq[3])
    yield CX(qq[1], qq[3])
    yield CX(qq[1], qq[4])
    yield CX(qq[2], qq[4])
    
    #adding qubit 3
    yield CX(qq[3], qq[5])
    
    #adding qubit 4
    yield CCX(qq[4], qq[5], qq[6])
    yield CX(qq[4], qq[5])
    
    #check if b1 is equal to 1 and store the result in the auxillary qubit
    yield CX(qq[6], qq[7])

def oracle():
    yield oracle_computation()
    yield Z(qq[7])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*(qq[:3]))
    yield X.on_each(*(qq[:3]))
    yield Z(qq[2]).controlled_by(qq[0], qq[1])
    yield X.on_each(*(qq[:3]))
    yield H.on_each(*(qq[:3]))


# the Grover algorithm
qq = cirq.LineQubit.range(8)
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:3])))

for i in range(1):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
    
circuit.append(cirq.measure(*qq[0:3], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)

print("Random guess probability:", 1./2**3)

# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.125
Probability of observing 010 :  0.5053
Probability of observing 101 :  0.4947


We observe that the Grover's Search outputs 101 and 010, which are the two colorings where there are 2 edges connecting vertices with different colors. 

In the remaining of this notebook, you will apply the same algorithm for other graphs. 

### Task 1

Implement the Grover algorithm for the following graph
<img src="../images/completebipartite.png" width="25%" align="center">

and check whether there is a coloring in which there are exactly four edges connecting vertices with a different color. Apply the oracle twice.

In [5]:
import cirq
from cirq import H, Z, X, inverse, I, CX, CCX
s = cirq.Simulator()

# 0-3: vertices
# ?-?: edge checking
# ?-?: the number
# ?-?: auxillary

def oracle_computation():
    #
    # your solution goes here
    # 
    yield CCX(qq[0],qq[2],qq[4])
    yield CCX(qq[0],qq[3],qq[5])
    yield CCX(qq[2],qq[1], qq[6])
    yield CCX(qq[1], qq[3],qq[7])
    
    yield CX(qq[4],qq[8])

    yield CCX(qq[5],qq[8],qq[9])
    yield CX(qq[5],qq[8])

    yield CCX(qq[6],qq[8],qq[9])
    yield CX(qq[6], qq[8])

    yield X(qq[10]).controlled_by(qq[7],qq[8],qq[9])
    yield CCX(qq[7],qq[8],qq[9])
    yield CX(qq[7], qq[8])

    yield X(qq[8])
    yield X(qq[9])
    yield X(qq[11]).controlled_by(*(qq[8:11]))


def oracle():    
    #
    # your solution goes here
    # 
    yield oracle_computation()
    yield Z(qq[10])
    yield inverse(oracle_computation())
    
def grover_diffusion():
    #
    # your solution goes here
    # 
    yield H.on_each(*qq[0:4])
    yield X.on_each(*qq[0:4])
    yield Z(qq[3]).controlled_by(*qq[0:3])
    yield H.on_each(*qq[0:4])
    yield X.on_each(*qq[0:4])

# the Grover algorithm
qq = cirq.LineQubit.range(12) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:4])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:4], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**4)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.0625
Probability of observing 0010 :  0.0378
Probability of observing 1111 :  0.4582
Probability of observing 0110 :  0.0319
Probability of observing 0000 :  0.0411
Probability of observing 1110 :  0.0439
Probability of observing 0001 :  0.0369
Probability of observing 1011 :  0.0416
Probability of observing 1001 :  0.0325
Probability of observing 1101 :  0.0392
Probability of observing 1100 :  0.0284
Probability of observing 0111 :  0.0394
Probability of observing 1010 :  0.0319
Probability of observing 1000 :  0.039
Probability of observing 0101 :  0.0282
Probability of observing 0100 :  0.0439
Probability of observing 0011 :  0.0261


[click for our solution](B04_Grover_Algorithm_For_Max-Cut_Problem_Solutions.ipynb#task1)

### Task 2

Implement the Grover Algorithm for the following graph
<img src="../images/finalgrover1.png" width="25%" align="center">

and check whether there is coloring with six or more edges connecting vertices with a different color. Apply the oracle twice. 

*Hint*: Note that you may check only two bits of the number.

In [8]:
import cirq
from cirq import H, Z, X, inverse, CX, CCX
s = cirq.Simulator()

#
# your solution
# 
def edge_check(v1, v2, edge):
    yield CCX(qq[v2], qq[v1], qq[edge])

def oracle_computation():
    yield edge_check(0, 1, 5)
    yield edge_check(0, 4, 6)
    yield edge_check(0, 3, 7)
    yield edge_check(1, 3, 8)
    yield edge_check(1, 4, 9)
    yield edge_check(3, 2, 10)
    yield edge_check(2, 4, 11)

    for i in range(5, 12):
        if i == 5:
            yield CX(qq[i], qq[12])

        elif i < 8:
            yield CCX(qq[i],qq[12],qq[13])
            yield CX(qq[i],qq[12])

        else:
            yield X(qq[14]).controlled_by(qq[i],qq[12],qq[13])
            yield CCX(qq[i],qq[12],qq[13])
            yield CX(qq[i],qq[12])

    yield X(qq[15]).controlled_by(qq[13],qq[14])

def oracle():
    yield oracle_computation()
    yield Z(qq[15])
    yield inverse(oracle_computation())

def grover_diffusion():
    yield H.on_each(*qq[0:5])
    yield X.on_each(*qq[0:5])
    yield Z(qq[4]).controlled_by(*qq[0:4])
    yield H.on_each(*qq[0:5])
    yield X.on_each(*qq[0:5])

qq = cirq.LineQubit.range(16) 
circuit = cirq.Circuit()
circuit.append(H.on_each(*(qq[0:5])))

for i in range(2):
    circuit.append(oracle())
    circuit.append(grover_diffusion())
circuit.append(cirq.measure(*qq[0:5], key='result'))

# determine the statistics of the measurements
trials_number = 10_000
samples = s.run(circuit, repetitions=trials_number)
result = samples.measurements["result"]

print("Random guess probability:", 1/2**5)
# create an histogram of the result
def bitstring(bits):
    return "".join(str(int(b)) for b in bits)
counts = samples.histogram(key="result",fold_func=bitstring)
for state, c in counts.items():
    print("Probability of observing", state, ": " ,c/trials_number)

Random guess probability: 0.03125
Probability of observing 01111 :  0.0369
Probability of observing 01010 :  0.0413
Probability of observing 01001 :  0.0388
Probability of observing 10000 :  0.0132
Probability of observing 10010 :  0.0405
Probability of observing 11111 :  0.214
Probability of observing 00110 :  0.0387
Probability of observing 01011 :  0.0119
Probability of observing 11101 :  0.0409
Probability of observing 00011 :  0.0379
Probability of observing 00010 :  0.0123
Probability of observing 11001 :  0.0106
Probability of observing 10110 :  0.0128
Probability of observing 01110 :  0.0128
Probability of observing 10100 :  0.0403
Probability of observing 00101 :  0.0378
Probability of observing 01101 :  0.0111
Probability of observing 10111 :  0.0376
Probability of observing 11000 :  0.0368
Probability of observing 00001 :  0.0149
Probability of observing 01100 :  0.0347
Probability of observing 11011 :  0.0387
Probability of observing 00100 :  0.0128
Probability of observing

[click for our solution](B04_Grover_Algorithm_For_Max-Cut_Problem_Solutions.ipynb#task2)

## Final remarks

We have successfully implemented the Grover's Search algorithm for the Max-Cut problem. Congratulations! However, there are several interesting additional facts.

Currently for each edge, we have a separate qubit. We could reuse a single qubit at the cost of additional quantum gates. While the circuit would be longer, we could save some qubits.

Next, for each task, the number of iterations was given. While for bipartiteness checking we know there are always exactly two answers, this may not be the case for the Max-Cut problem. Fortunately, there is a *quantum counting algorithm*, which can efficiently determine the number of solutions and hence the number of iterations.